In [1]:
import os

class Hooklog3Xml:
    path = None    
    length = None
    li = None
    
    def __init__(self, path):
        self.path = path
        self.li = list()
        self.length = 0
        
        self.__parseHooklogXml()

    def isExist(self):
        if os.path.isfile(self.path):
            return True
        else:
            return False
    
    def __len__(self):
        return self.length
    
    # private functions
    def __parseHooklogXml(self):
        if self.isExist():
            handle = open(self.path, 'rb')
            
            while 1:
                (tick, api, key_val_list) = self.__getNextCallXml(handle)
                if tick == 0:
                    break # end of file or fail to read
                self.li.append((tick, api, key_val_list))
                self.length += 1
            if handle: handle.close()
            
            self.li.sort(key = lambda tup: tup[0]) # sort by tick
        else:
            pass
    
    def __getNextCallXml(self, handle):
        tick = 0
        api = ''
        key_val_list = list()
        
        while 1:
            line = handle.readline().decode("ascii") # MIKE: 20170616, for python 3
            if not line: break # end of file
            if line[0] == '#': # start a new call
                tick = line[1:].strip()
                api = handle.readline().decode("ascii").strip() # MIKE: 20170616, for python 3
                
                # hack
                if api.startswith('=Reg'): api = api[1:]
                
                # rest of parameters
                while 1:
                    pos = handle.tell()
                    key_value = handle.readline().decode("ascii").strip() # MIKE: 20170616, for python 3
                    if not key_value: break
                    if key_value[0] == '#': # ouch
                        handle.seek(pos)
                        break
                    else:
                        key_val_list.append(key_value)
                break # get one call
            else:
                continue
        
        return (tick, api, key_val_list)
    
    # public functions
    def printXml(self):
        print("<?xml version=\"1.0\"?>")
        print("<Profile>")
        print("<Meta>")
        print("<Hash>%s</Hash>" % (self.path.split('_')[0].split('/')[-1]))
        print("<Process_Id>%s</Process_Id>" % (self.path.split('_')[1].split('.')[0]))
        print("</Meta>")
        
        print("<Execution>")
        for i in range(self.length):
            (tick, api, key_val_list) = self.li[i]
            o = "<" + api
            for k_v in key_val_list:
                tokens = k_v.split('=')
                if len(tokens) > 2: print("ERROR!!! Has a = in value.")
                k = tokens[0]
                v = tokens[1]
                o += (" "+k+"=\""+v+"\"")
            o += " Time=\""+ tick +"\" />"
            print(o)
        print("</Execution>")
        print("</Profile>")